In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     5000              1  
1         1-9999                      N   108590              1  
2              0                      N     5000              0  
3    10000-24999                      N     6692              1  
4  100000-499999                      N   142590              1

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN' , 'NAME'], axis=1)
application_df.head()

APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0              T10       Independent          C1000    ProductDev   
1               T3       Independent          C2000  Preservation   
2               T5  CompanySponsored          C3000    ProductDev   
3               T3  CompanySponsored          C2000  Preservation   
4               T3       Independent          C1000     Heathcare   

   ORGANIZATION  STATUS     INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  \
0   Association       1              0                      N     5000   
1  Co-operative       1         1-9999                      N   108590   
2   Association       1              0                      N     5000   
3         Trust       1    10000-24999                      N     6692   
4         Trust       1  100000-499999                      N   142590   

   IS_SUCCESSFUL  
0              1  
1              1  
2              0  
3              1  
4              1

In [3]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
cutoff_value = 500
application_types_to_replace = []

# Get the value counts of each application type
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Create a list of application types to be replaced
for app_type, count in application_type_counts.items():
    if count < cutoff_value:
        application_types_to_replace.append(app_type)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

# You may find it helpful to look at CLASSIFICATION value counts >1
filtered_classification_counts = classification_counts[classification_counts > 1]

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 100
classifications_to_replace = []

# Get the value counts of each classification
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Create a list of classifications to be replaced
for cls, count in classification_counts.items():
    if count < cutoff_value:
        classifications_to_replace.append(cls)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [5]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()
     

STATUS  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0       1     5000              1                       0   
1       1   108590              1                       0   
2       1     5000              0                       0   
3       1     6692              1                       0   
4       1   142590              1                       0   

   APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                     1                     0                    0   
1                     0                     0                    1   
2                     0                     0                    0   
3                     0                     0                    1   
4                     0                     0                    1   

   APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  ...  \
0                    0                    0                    0  ...   
1                    0                    0                    0  ...   
2                    0                    1                    0  ...   
3                    0                    0                    0  ...   
4                    0                    0                    0  ...   

   INCOME_AMT_1-9999  INCOME_AMT_10000-24999  INCOME_AMT_100000-499999  \
0                  0                       0                         0   
1                  1                       0                         0   
2                  0                       0                         0   
3                  0                       1                         0   
4                  0                       0                         1   

   INCOME_AMT_10M-50M  INCOME_AMT_1M-5M  INCOME_AMT_25000-99999  \
0                   0                 0                       0   
1                   0                 0                       0   
2                   0                 0                       0   
3                   0                 0                       0   
4                   0                 0                       0   

   INCOME_AMT_50M+  INCOME_AMT_5M-10M  SPECIAL_CONSIDERATIONS_N  \
0                0                  0                         1   
1                0                  0                         1   
2                0                  0                         1   
3                0                  0                         1   
4                0                  0                         1   

   SPECIAL_CONSIDERATIONS_Y  
0                         0  
1                         0  
2                         0  
3                         0  
4                         0  

[5 rows x 50 columns]

In [6]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values


# drop 'IS SUCCESSFUL'
X = application_df.drop('IS_SUCCESSFUL', axis=1).values

  

In [7]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [8]:

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len( X_train_scaled[0])
hidden_nodes_layer1=14
hidden_nodes_layer2=7
nn = tf.keras.models.Sequential()

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 14)                700       
_________________________________________________________________
dense_4 (Dense)              (None, 7)                 105       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 8         
Total params: 813
Trainable params: 813
Non-trainable params: 0
_________________________________________________________________


In [14]:

# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,validation_split=0.15, epochs=100)

Train on 21865 samples, validate on 3859 samples
Epoch 1/100
21865/21865 [==============================] - 1s 61us/sample - loss: 0.6152 - accuracy: 0.6913 - val_loss: 0.5635 - val_accuracy: 0.7336
Epoch 2/100
21865/21865 [==============================] - 1s 37us/sample - loss: 0.5640 - accuracy: 0.7249 - val_loss: 0.5505 - val_accuracy: 0.7378
Epoch 3/100
21865/21865 [==============================] - 1s 38us/sample - loss: 0.5574 - accuracy: 0.7264 - val_loss: 0.5488 - val_accuracy: 0.7334
Epoch 4/100
21865/21865 [==============================] - 1s 40us/sample - loss: 0.5545 - accuracy: 0.7275 - val_loss: 0.5446 - val_accuracy: 0.7383
Epoch 5/100
21865/21865 [==============================] - 1s 39us/sample - loss: 0.5525 - accuracy: 0.7284 - val_loss: 0.5440 - val_accuracy: 0.7375
Epoch 6/100
21865/21865 [==============================] - 1s 37us/sample - loss: 0.5518 - accuracy: 0.7275 - val_loss: 0.5442 - val_accuracy: 0.7357
Epoch 7/100
21865/21865 [==========================

Epoch 55/100
21865/21865 [==============================] - 1s 46us/sample - loss: 0.5418 - accuracy: 0.7344 - val_loss: 0.5428 - val_accuracy: 0.7378
Epoch 56/100
21865/21865 [==============================] - 1s 46us/sample - loss: 0.5413 - accuracy: 0.7351 - val_loss: 0.5430 - val_accuracy: 0.7354
Epoch 57/100
21865/21865 [==============================] - 1s 44us/sample - loss: 0.5413 - accuracy: 0.7344 - val_loss: 0.5448 - val_accuracy: 0.7372
Epoch 58/100
21865/21865 [==============================] - 1s 43us/sample - loss: 0.5414 - accuracy: 0.7348 - val_loss: 0.5439 - val_accuracy: 0.7385
Epoch 59/100
21865/21865 [==============================] - 1s 43us/sample - loss: 0.5412 - accuracy: 0.7347 - val_loss: 0.5416 - val_accuracy: 0.7367
Epoch 60/100
21865/21865 [==============================] - 1s 43us/sample - loss: 0.5412 - accuracy: 0.7346 - val_loss: 0.5413 - val_accuracy: 0.7388
Epoch 61/100
21865/21865 [==============================] - 1s 52us/sample - loss: 0.5409 - ac

In [12]:

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 0s - loss: 0.5266 - accuracy: 0.7287
Loss: 0.5526952306800264, Accuracy: 0.7287463545799255
